In [1]:
import sys
import os 
from pathlib import Path
import pandas as pd
import json
import numpy as np

project_path = Path(os.path.dirname(os.path.realpath("__file__"))).parent

project_path = Path(os.path.dirname(os.path.realpath("__file__"))).parent

from scripts.utils import log_df

from scripts.catalog_utils import (
format_ngc_catalog,
format_best_500_catalog,
format_best_500_catalog_M, 
merge_type_constellation, 
catalog_columns,
decToHMS,
convert_to_hh_mm_ss,
convert_to_hh_mm_ss_d
)

In [2]:
mike_draft_path = project_path/'data'/'draft'/'mike_catalog_draft.csv'

best_500dso_path_draft = project_path/'data'/'draft'/'Best_500_Deep_Sky_Objects-draft.csv'
# final best_500dso file
best_500dso_path = project_path/'data'/'draft'/'Best_500_Deep_Sky_Objects-util.csv'

# tmp file for control
# temp best_500dso files
best_500dso_path_tmp = project_path/'data'/'draft'/'Best_500_Deep_Sky_Objects-tmp.csv'
best_500dso_path_merge = project_path/'data'/'draft'/'Best_500_Deep_Sky_Objects-merge.csv'

# openngc file
openngc_path = project_path / 'raw_data' / 'OpenNGC.csv'

# temp openngc files
ngc_draft_path_tmp1 = project_path/'data'/'draft'/'openngc_catalog_draft_tmp1.csv'
ngc_draft_path_tmp2 = project_path/'data'/'draft'/'openngc_catalog_draft_tmp2.csv'

# final ngc_draft file
ngc_draft_path = project_path/'data'/'draft'/'openngc_catalog_draft.csv'


# type files
best_500dso_type_path = project_path/'raw_data'/'Best_500_types.csv'
openngc_types_path = project_path / 'raw_data' / 'OpenNGC_types.csv' 
constellation_path = project_path/'raw_data'/'constellations.csv'

# others files
constellation_path = project_path/'raw_data'/'constellations.csv'
solar_system_path = project_path/'raw_data'/'moon_planets.csv'
hyg_path = project_path / 'raw_data' / 'HYG-Database' / 'hyg' / 'v3' / 'hyg_v35.csv' 

# solar_system files
solar_system_catalog_path = project_path/'data'/'catalogs'/'moon_planets.csv'
solar_system_catalog_json_path = project_path/'data'/'catalogs'/'moon_planets.json'

# final dso_catalog file
dso_catalog_path = project_path/'data'/'catalogs'/'openngc_dso.csv'
dso_catalog_json_path = project_path/'data'/'catalogs'/'dso_catalog.json'
stars_catalog_path = project_path/'data'/'catalogs'/'hyg_stars.csv'

demo_path = project_path/'data'/'draft'/'demo_catalog.json'


In [3]:
openngc_columns = [
    'Name', 'Type', 'RA', 'Dec', 'Const', 'MajAx', 'MinAx', 'V-Mag', 
    'M', 'NGC', 'IC',  'Common names'
]
mike_columns = [
    'Catalog', 
    'Name', 'Name (no leading zero)', 'Name (final)', 'Common Name', 'NGC/ID', 
    "Width (')", "Height (')", 'Mag', 'Type', 'RA', 'Dec'
]

best500_columns = [
    'CONST.', 'TYPE', 'NGC DESIGNATION', 'OTHER NAME(S)', 'R.A.', 'DEC.', 'MAG', 'MAX SZ', 'MIN SZ'
]

best500_columns_modif = [
    'CONST.', 'Types_OpenNGC', 'NGC DESIGNATION', 'OTHER NAME(S)', 'Name normalized', 'R.A.','DEC.','MAG','MAX SZ','MIN SZ','Name catalog','Name number','Best_500'
]
hyg_columns = [
    'hip',  'hd', 'hr', 
    'proper', 'ra', 'dec', 'mag', 'con'
]


## Format best_500dso to file compatible with mergin with ngc data

In [4]:
best500_df = pd.read_csv(best_500dso_path_draft, usecols=best500_columns)
best500_df = format_best_500_catalog(best500_df)

# tmp file for control
#best500_df.to_csv(best_500dso_path_tmp, index=False)

# Extract non-duplicate values of the column
types_Best500 = best500_df['TYPE'].drop_duplicates()
# Convert non-duplicate values to a DataFrame
types_Best500_df = pd.DataFrame(types_Best500, columns=['TYPE'])

# Save  best_500dso TYPE to a CSV file
types_Best500_df.to_csv(best_500dso_type_path, index=False)

best500_2_df = format_best_500_catalog_M(best500_df)
best500_2_df.to_csv(best_500dso_path, index=False)


## Create DSO Draft Catalog

In [5]:
ngc_df = pd.read_csv(openngc_path, usecols=openngc_columns, dtype={'M': pd.Int64Dtype()})
ngc_df = ngc_df[ngc_df['Type'] != 'Dup']
ngc_df.loc[ngc_df['M'] == 65, 'Common names'] = 'Leo Triplet'

log_df(ngc_df)
# (13340, 12)

(13340, 12)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,IC,Common names
0,IC0001,**,00:08:27.05,+27:43:03.6,Peg,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1,IC0002,G,00:11:00.88,-12:49:22.3,Cet,0.98,0.32,NaN,<NA>,NaN,NaN,NaN
2,IC0003,G,00:12:06.09,-00:24:54.8,Psc,0.93,0.67,NaN,<NA>,NaN,NaN,NaN
3,IC0004,G,00:13:26.94,+17:29:11.2,Peg,1.17,0.84,NaN,<NA>,NaN,NaN,NaN
4,IC0005,G,00:17:34.93,-09:32:36.1,Cet,0.99,0.66,NaN,<NA>,NaN,NaN,NaN


In [6]:
df = format_ngc_catalog(ngc_df)

log_df(df)

# (13340, 18)

(13340, 18)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,IC,Common names,Name catalog,Name number (with zeros),Name normalized,IC name,M name,Name number
0,IC0001,**,00:08:27.05,+27:43:03.6,Peg,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,IC,0001,IC 1,IC 1,NaN,1
1,IC0002,G,00:11:00.88,-12:49:22.3,Cet,0.98,0.32,NaN,<NA>,NaN,NaN,NaN,IC,0002,IC 2,IC 2,NaN,2
2,IC0003,G,00:12:06.09,-00:24:54.8,Psc,0.93,0.67,NaN,<NA>,NaN,NaN,NaN,IC,0003,IC 3,IC 3,NaN,3
3,IC0004,G,00:13:26.94,+17:29:11.2,Peg,1.17,0.84,NaN,<NA>,NaN,NaN,NaN,IC,0004,IC 4,IC 4,NaN,4
4,IC0005,G,00:17:34.93,-09:32:36.1,Cet,0.99,0.66,NaN,<NA>,NaN,NaN,NaN,IC,0005,IC 5,IC 5,NaN,5


In [7]:
df[df['Name']=='IC0359A'][['Name', 'M',  'Name catalog', 'Name number (with zeros)', 'Name number']]

,Name,M,Name catalog,Name number (with zeros),Name number
371,IC0359A,<NA>,IC,0359A,359


In [8]:
df[df['Name']=='IC4715'][['Name', 'M',  'Name catalog','Name number (with zeros)', 'Name number']]

,Name,M,Name catalog,Name number (with zeros),Name number
4903,IC4715,24,M,4715,24


In [9]:
#mike_file is file containing the large DSO
#mike file has bee restored and updated

draft_df = pd.read_csv(mike_draft_path, dtype={'Name number': pd.Int64Dtype()})

merge_df = df.merge(draft_df,
                    left_on='Name normalized', right_on='Name normalized',
                    how='outer' , suffixes=[None, '_MIKE'])


merge_df['Name number'] = merge_df['Name number'].astype(pd.Int64Dtype())

log_df(merge_df)
# (13349, 35)

# tmp file for control
#merge_df.to_csv(ngc_draft_path_tmp1, index=False)





(13349, 35)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,...,Mag,RA_MIKE,Dec_MIKE,Width ('),Height ('),IC_MIKE,IC name_MIKE,NGC_MIKE,NGC name,mike
0,B033,DrkN,05:40:59.0,-02:27:30,Ori,6.0,4.0,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C014,*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,<NA>,NaN,...,4.3,02:20:00,57.13,30.0,30.0,NaN,NaN,NaN,NaN,True
2,C041,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,<NA>,NaN,...,1,04:27:00,16.00,330.0,330.0,NaN,NaN,NaN,NaN,True
3,C009,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,<NA>,NaN,...,7.7,22:56:48,62.62,50.0,10.0,NaN,NaN,NaN,NaN,True
4,C099,DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,<NA>,NaN,...,NaN,21:40:00,41.00,300.0,300.0,NaN,NaN,NaN,NaN,True


In [10]:
merge_df[merge_df['Name']=='IC4715'][['Name', 'Name number']]

,Name,Name number
5261,IC4715,24
5262,IC4715,24


In [21]:
# best500 is file containing the bedt 500 DSO

best500_df = pd.read_csv(best_500dso_path, usecols=best500_columns_modif)
best500_df.rename(columns={'MAG': 'MAG_500'}, inplace=True)

merge2_df = merge_df.merge(best500_df,
                    left_on='Name normalized', right_on='Name normalized',
                    how='outer' , suffixes=[None, '_500'])

merge2_df['Name number'] = merge2_df['Name number'].astype(pd.Int64Dtype())

log_df(merge2_df)
# (13366, 47)

# tmp file for control
#merge2_df.to_csv(best_500dso_path_merge, index=False)

(13366, 47)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,...,OTHER NAME(S),R.A.,DEC.,MAG_500,MAX SZ,MIN SZ,Name catalog_500,Name number_500,Types_OpenNGC,Best_500
0,B033,DrkN,05:40:59.0,-02:27:30,Ori,6.0,4.0,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,"Forms a pairing with NGC6520 open cluster, INK...",18 03.0,-27 52,NaN,6.0,4.0,B,86.0,DrkN,True
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,on northside of M24,18 15.6,-18 14,NaN,15.0,9.0,B,92.0,DrkN,True
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,on northside of M24,18 16.9,-18 04,NaN,8.0,3.0,B,93.0,DrkN,True
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,,04 44.5,+42 04,9.8,12.0,12.0,Berkeley,68.0,OCl,True


In [12]:
filter_df = merge2_df.copy()

size_limit = 15
mag_limit = 10

size_bool = (filter_df['MajAx'] >= size_limit) | (filter_df['MinAx'] >= size_limit)

size_b500 = (filter_df['MAX SZ'] >= size_limit) | (filter_df['MIN SZ'] >= size_limit)

# filter_df = filter_df[size_bool]
filter_df = filter_df[(size_bool) | (filter_df['mike'] == True) | (size_b500)]

filter_df = filter_df[(filter_df['V-Mag'] <= mag_limit) | (filter_df['MAG_500'] <= str(mag_limit)) | filter_df['V-Mag'].isna() | filter_df['MAG_500'].isna()]
# filter_df = filter_df[filter_df['V-Mag'] <= mag_limit]
filter_df['Notes'] = 'large_dso'

filter_df = filter_df[(filter_df['Type'] != '**')]
filter_df = filter_df[(filter_df['Type'] != '*')]


log_df(filter_df)
# (244, 48) size 15 + NA, mag 10,  mike, best 500

# (208, 35) size 15 + NA, mag 10,  mike
# (137, 35) size 15 + NA, mag 10
# (122, 35) size 15,  mag 10, mike
# (66, 35) size 15, mag 10
# (201, 35) size 15 + NA, mag 9,  mike

(244, 48)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,...,R.A.,DEC.,MAG_500,MAX SZ,MIN SZ,Name catalog_500,Name number_500,Types_OpenNGC,Best_500,Notes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,18 15.6,-18 14,NaN,15.0,9.0,B,92.0,DrkN,True,large_dso
6,C014,*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
7,C041,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
8,C009,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
9,C099,DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso


In [13]:
filter2_df = merge2_df.copy()

size_limit = 1
mag_limit = 10

size2_bool = (filter2_df['MajAx'] > size_limit) | (filter2_df['MinAx'] > size_limit)

size2_b500 = (filter_df['MAX SZ'] >= size_limit) | (filter_df['MIN SZ'] >= size_limit)

filter2_df = filter2_df[((size2_bool) | (size2_b500)) & (filter2_df['mike'].isna())]
filter2_df = filter2_df[(filter2_df['V-Mag'] <= mag_limit) | (filter2_df['MAG_500'] <= str(mag_limit))]

filter2_df['Notes'] = 'small_dso'
 
log_df(filter2_df)
# (403, 48) 

(403, 48)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,...,R.A.,DEC.,MAG_500,MAX SZ,MIN SZ,Name catalog_500,Name number_500,Types_OpenNGC,Best_500,Notes
11,Cl399,*Ass,19:25:24.0,+20:11:00,Vul,70.00,NaN,3.60,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,small_dso
12,ESO351-030,G,01:00:09.35,-33:42:32.5,Scl,15.26,15.26,8.60,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,small_dso
13,ESO356-004,G,02:39:59.33,-34:26:57.1,For,12.88,10.47,7.40,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,small_dso
14,ESO056-115,G,05:23:34.5,-69:45:22,Dor,646.00,550.00,0.29,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,small_dso
16,H20,OCl,19:53:06.0,+18:20:00,Sge,5.10,NaN,7.70,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,small_dso


In [14]:
filter3_df = merge2_df.copy()

size_limit = 0.5
mag_limit = 12

size3_bool = (filter3_df['MajAx'] > size_limit) | (filter3_df['MinAx'] > size_limit)
size3_b500 = (filter3_df['MAX SZ'] >= size_limit) | (filter3_df['MIN SZ'] >= size_limit)

filter3_df = filter3_df[((size3_bool) | (size3_b500)) & (filter3_df['mike'].isna())]
filter3_df = filter3_df[filter3_df['Common names'].notna() | filter3_df['OTHER NAME(S)'].notna()]
filter3_df = filter3_df[(filter3_df['V-Mag'] <= mag_limit) | (filter3_df['MAG_500'] <= str(mag_limit))]

filter3_df['Notes'] = 'tiny_dso'
 
log_df(filter3_df)
# (373, 48)

(373, 48)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,...,R.A.,DEC.,MAG_500,MAX SZ,MIN SZ,Name catalog_500,Name number_500,Types_OpenNGC,Best_500,Notes
11,Cl399,*Ass,19:25:24.0,+20:11:00,Vul,70.00,NaN,3.60,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tiny_dso
12,ESO351-030,G,01:00:09.35,-33:42:32.5,Scl,15.26,15.26,8.60,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tiny_dso
13,ESO356-004,G,02:39:59.33,-34:26:57.1,For,12.88,10.47,7.40,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tiny_dso
14,ESO056-115,G,05:23:34.5,-69:45:22,Dor,646.00,550.00,0.29,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tiny_dso
24,IC0010,G,00:20:17.34,+59:18:13.6,Cas,6.76,6.03,10.35,<NA>,NaN,...,00 20.4,+59 18,13.3,6.4,5.3,IC,10.0,G,True,tiny_dso


In [15]:
combine1_df = pd.concat([filter_df, filter2_df])
combine_df = pd.concat([combine1_df, filter3_df])

cols = [col for col in combine_df.columns if col != 'Notes']
combine_df.drop_duplicates(inplace=True, subset=cols, keep='first')

log_df(combine_df)
# (774, 48)

(774, 48)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,...,R.A.,DEC.,MAG_500,MAX SZ,MIN SZ,Name catalog_500,Name number_500,Types_OpenNGC,Best_500,Notes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,18 15.6,-18 14,NaN,15.0,9.0,B,92.0,DrkN,True,large_dso
6,C014,*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
7,C041,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
8,C009,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
9,C099,DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso


In [16]:
# Apply the function to extract the number from the 'Name' column
combine_df['No Old Value'] = (combine_df['Name'].isna()) & (combine_df['Best_500'] == True)

# tmp file for control
#combine_df.to_csv(ngc_draft_path_tmp2, index=False)

# add only data form Best_500
combine_df['Type'] = combine_df.apply(lambda row:row['Types_OpenNGC']  if (row['No Old Value']) else row['Type'], axis=1)
combine_df['RA'] = combine_df.apply(lambda row:convert_to_hh_mm_ss(row['R.A.'])  if (row['No Old Value']) else row['RA'], axis=1)
combine_df['Dec'] = combine_df.apply(lambda row:convert_to_hh_mm_ss_d(row['DEC.'])  if (row['No Old Value']) else row['Dec'], axis=1)
combine_df['Const'] = combine_df.apply(lambda row:row['CONST.']  if (row['No Old Value']) else row['Const'], axis=1)
combine_df['MajAx'] = combine_df.apply(lambda row:row['MAX SZ']  if (row['No Old Value']) else row['MajAx'], axis=1)
combine_df['MinAx'] = combine_df.apply(lambda row:row['MIN SZ']  if (row['No Old Value']) else row['MinAx'], axis=1)
combine_df['V-Mag'] = combine_df.apply(lambda row:row['MAG_500']  if (row['No Old Value']) else row['V-Mag'], axis=1)
combine_df['NGC'] = combine_df.apply(lambda row:row['NGC DESIGNATION']  if (row['No Old Value']) else row['NGC'], axis=1)
combine_df['Common names'] = combine_df.apply(lambda row:row['OTHER NAME(S)']  if (row['No Old Value']) else row['Common names'], axis=1)
combine_df['Name catalog'] = combine_df.apply(lambda row:row['Name catalog_500']  if (row['No Old Value']) else row['Name catalog'], axis=1)
combine_df['Name number'] = combine_df.apply(lambda row:row['Name number_500']  if (row['No Old Value']) else row['Name number'], axis=1)
combine_df['Name'] = combine_df.apply(lambda row:row['Name normalized']  if (row['No Old Value']) else row['Name'], axis=1)

combine_df.drop(columns=['No Old Value'], inplace=True)

log_df(combine_df)
# (774, 48)

combine_df.to_csv(ngc_draft_path, index=False)

(774, 48)


## Create DSO  Catalog

In [17]:
draft_df = pd.read_csv(ngc_draft_path, dtype={'Name number': pd.Int64Dtype()})
log_df(draft_df)
# (774, 48)

(774, 48)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,...,R.A.,DEC.,MAG_500,MAX SZ,MIN SZ,Name catalog_500,Name number_500,Types_OpenNGC,Best_500,Notes
0,B 92,DrkN,18:15:06,-18:14:00.0,Sgr,15.0,9.0,NaN,NaN,B92,...,18 15.6,-18 14,NaN,15.0,9.0,B,92.0,DrkN,True,large_dso
1,C014,*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
2,C041,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
3,C009,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso
4,C099,DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso


In [18]:
draft_df.columns

Index(['Name', 'Type', 'RA', 'Dec', 'Const', 'MajAx', 'MinAx', 'V-Mag', 'M',
       'NGC', 'IC', 'Common names', 'Name catalog', 'Name number (with zeros)',
       'Name normalized', 'IC name', 'M name', 'Name number', 'Catalog',
       'Name_MIKE', 'Name (no leading zero)', 'Name (final)', 'NGC/ID',
       'Type_MIKE', 'Common Name', 'Mag', 'RA_MIKE', 'Dec_MIKE', 'Width (')',
       'Height (')', 'IC_MIKE', 'IC name_MIKE', 'NGC_MIKE', 'NGC name', 'mike',
       'CONST.', 'NGC DESIGNATION', 'OTHER NAME(S)', 'R.A.', 'DEC.', 'MAG_500',
       'MAX SZ', 'MIN SZ', 'Name catalog_500', 'Name number_500',
       'Types_OpenNGC', 'Best_500', 'Notes'],
      dtype='object')

add name columns that have names from NGC and Mike

In [22]:
tmp_df =  draft_df.copy()
tmp_df.dropna(subset=['Name'], inplace=True)

for index, row in tmp_df.iterrows():
    # create string with multiple names
    names = set([row['Name normalized'],
                 row['IC name'], row['M name'],
                 row['Name (no leading zero)'], row['IC name_MIKE'], row['NGC name']])
    names.remove(row['Name normalized'])
    names = [name for name in names if pd.notna(name)]
    names.sort()
    tmp_df.at[index, 'Alternate Names'] = ', '.join(names)

    # create string with multiple common names
    common_names = set([row['Common names'], row['Common Name']])
    common_names = [name for name in common_names if pd.notna(name)]
    common_names.sort()
    tmp_df.at[index, 'Common Names'] = ', '.join(common_names)

log_df(tmp_df)
# (774, 50)

(774, 50)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,M,NGC,...,MAG_500,MAX SZ,MIN SZ,Name catalog_500,Name number_500,Types_OpenNGC,Best_500,Notes,Alternate Names,Common Names
0,B 92,DrkN,18:15:06,-18:14:00.0,Sgr,15.0,9.0,NaN,NaN,B92,...,NaN,15.0,9.0,B,92.0,DrkN,True,large_dso,,on northside of M24
1,C014,*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso,,"Double Cluster, h & chi Persei"
2,C041,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso,,Hyades
3,C009,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso,,Cave Nebula
4,C099,DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,large_dso,NTH CS,"Coalsack Nebula, Coalsack Northern"


In [23]:
tmp_df = tmp_df[
    [
        'Name normalized', 'Alternate Names', 'Common Names', 
        'Type', 'RA', 'Dec', 'Const', 'MajAx', 'MinAx', 'V-Mag',  
        'Name catalog', 'Name number', "Width (')", "Height (')", 'Notes'
    ]
].copy()

tmp_df.rename(columns={'Name normalized': 'Name'},  inplace=True)

tmp_df = tmp_df.sort_values(['Name catalog', 'Name number'])

log_df(tmp_df)
# (774, 15)

(774, 15)


,Name,Alternate Names,Common Names,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,Name catalog,Name number,Width ('),Height ('),Notes
0,B 92,,on northside of M24,DrkN,18:15:06,-18:14:00.0,Sgr,15.0,9.0,NaN,B,92,NaN,NaN,large_dso
3,C 9,,Cave Nebula,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,C,9,50.0,10.0,large_dso
1,C 14,,"Double Cluster, h & chi Persei",*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,C,14,30.0,30.0,large_dso
2,C 41,,Hyades,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,C,41,330.0,330.0,large_dso
4,C 99,NTH CS,"Coalsack Nebula, Coalsack Northern",DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,C,99,300.0,300.0,large_dso


delete rows with duplicate names

In [24]:
dups = tmp_df[tmp_df.duplicated(subset=['Name'])]['Name'].values

In [25]:
dup_df = tmp_df[tmp_df['Name'].isin(dups)][['Name','Alternate Names', 'Common Names', 'Type']]
dup_df

,Name,Alternate Names,Common Names,Type
4,C 99,NTH CS,"Coalsack Nebula, Coalsack Northern",DrkN
5,C 99,,Coalsack Nebula,DrkN
37,IC 405,C 31,Flaming Star Nebula,Neb
38,IC 405,,"Flaming Star Nebula, Flaming Star nebula",Neb
22,IC 2391,C 85,"Omicron Vel Cluster, omi Vel Cluster",OCl
23,IC 2391,,omi Vel Cluster,OCl
25,IC 2602,C 102,"Theta Car Cluster, tet Car Cluster",OCl
26,IC 2602,,"Southern Pleiades, tet Car Cluster",OCl
65,M 16,NGC 6611,"Eagle Nebula, Eagle nebula",Neb
66,M 16,NGC 6611,"Eagle Nebula, Eagle nebula",Neb


In [26]:
ngc_catalog_df = tmp_df.copy()

def get_index_by_name(name):
    try:
        return dup_df[dup_df['Name'] == name].index[0]
    except:
        print(name, 'not found')
 

def get_index_by_common_name(name):
    try:
        return dup_df[dup_df['Common Names'] == name].index[0]
    except:
        print(name, 'not found..')
 
names = [
    'Coalsack Nebula',
    'Flaming Star Nebula, Flaming Star nebula', 
    'omi Vel Cluster',
    'Southern Pleiades, tet Car Cluster',
    'Small Sgr Star Cloud',
    'Eagle Nebula, Eagle nebula',
    'M  25',
    'M  67',
    'Filamentary nebula, Veil Nebula,Filamentary Nebula,Western Veil',
    'Eastern Veil,Network Nebula, Network nebula'
]
indexes = [get_index_by_common_name(name) for name in names]

ngc_catalog_df.drop(indexes, inplace=True)

ngc_catalog_df.at[get_index_by_name('IC 2602'), 'Common Names'] = 'Theta Car Cluster, Southern Pleiades'

log_df(ngc_catalog_df)
# (764, 15)

(764, 15)


,Name,Alternate Names,Common Names,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,Name catalog,Name number,Width ('),Height ('),Notes
0,B 92,,on northside of M24,DrkN,18:15:06,-18:14:00.0,Sgr,15.0,9.0,NaN,B,92,NaN,NaN,large_dso
3,C 9,,Cave Nebula,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,C,9,50.0,10.0,large_dso
1,C 14,,"Double Cluster, h & chi Persei",*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,C,14,30.0,30.0,large_dso
2,C 41,,Hyades,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,C,41,330.0,330.0,large_dso
4,C 99,NTH CS,"Coalsack Nebula, Coalsack Northern",DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,C,99,300.0,300.0,large_dso


In [27]:
ngc_catalog_df[ngc_catalog_df.duplicated(subset=['Name'])]['Name'].values

array([], dtype=object)

In [28]:
merge_df = merge_type_constellation(ngc_catalog_df, openngc_types_path, constellation_path)

log_df(merge_df)
# (228, 16)

(764, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
0,B 92,,on northside of M24,18:15:06,-18:14:00.0,15.0,9.0,NaN,B,92,Dark Nebula,nebulae,Sagittarius,NaN,NaN,large_dso
1,C 9,,Cave Nebula,22:57:54.0,+62:31:06,50.0,30.0,NaN,C,9,HII Ionized region,nebulae,Cepheus,50.0,10.0,large_dso
2,C 14,,"Double Cluster, h & chi Persei",02:20:42.0,+57:08:15,50.0,50.0,NaN,C,14,Association of stars,stars,Perseus,30.0,30.0,large_dso
3,C 41,,Hyades,04:26:54.0,+15:52:00,329.0,NaN,NaN,C,41,Open Cluster,clusters,Taurus,330.0,330.0,large_dso
4,C 99,NTH CS,"Coalsack Nebula, Coalsack Northern",12:31:19.0,-63:44:36,NaN,NaN,NaN,C,99,Dark Nebula,nebulae,Crux,300.0,300.0,large_dso


In [29]:
merge_df.to_csv(dso_catalog_path, index=False)

## create HYG stars catalogue

In [30]:
types = {'hip': pd.Int64Dtype(), 'hd': pd.Int64Dtype(), 'hr': pd.Int64Dtype()}
df = pd.read_csv(hyg_path, dtype=types, usecols=hyg_columns)
log_df(df)

(119614, 8)


,hip,hd,hr,proper,ra,dec,mag,con
0,<NA>,<NA>,<NA>,Sol,0.000000,0.000000,-26.70,NaN
1,1,224700,<NA>,NaN,0.000060,1.089009,9.10,Psc
2,2,224690,<NA>,NaN,0.000283,-19.498840,9.27,Cet
3,3,224699,<NA>,NaN,0.000335,38.859279,6.61,And
4,4,224707,<NA>,NaN,0.000569,-51.893546,8.06,Phe


In [31]:
cons_df = pd.read_csv(constellation_path, usecols=['Abbreviations IAU', 'name'])
cons_df.rename(columns={'name': 'Constellation'}, inplace=True)
log_df(cons_df)

(90, 2)


,Abbreviations IAU,Constellation
0,And,Andromeda
1,Ant,Antlia
2,Aps,Apus
3,Aqr,Aquarius
4,Aql,Aquila


In [32]:
# Make a copy of the DataFrame
df_copy = df.copy()

# Filter out rows where 'proper' is not NaN and 'mag' is less than or equal to 2.1
brightest_stars_df = df_copy[df_copy['proper'].notna() & (df_copy['mag'] <= 2.1)]

# Get the list of constellations that are represented in the filtered DataFrame
represented_constellations = brightest_stars_df['con'].unique()

# Get the list of constellations in the original DataFrame
all_constellations = df_copy['con'].unique()

# Find constellations that are not represented in the filtered DataFrame
missing_constellations = set(all_constellations) - set(represented_constellations)

# For each missing constellation, include the brightest star from that constellation
for constellation in missing_constellations:
    brightest_star = df_copy[df_copy['con'] == constellation].nsmallest(1, 'mag')
    brightest_stars_df = pd.concat([brightest_stars_df, brightest_star])

# Now brightest_stars_df contains the brightest stars for each constellation
filter_df = brightest_stars_df.copy()

filter_df.drop([0], inplace=True)

filter_df.sort_values(by=['mag'], inplace=True)

filter_df.reset_index(inplace=True)
#filter_df.drop(list(range(50,61)), inplace=True)
del filter_df['index']

log_df(filter_df)

# (118, 8)

(118, 8)


,hip,hd,hr,proper,ra,dec,mag,con
0,32349,48915,2491,Sirius,6.752481,-16.716116,-1.44,CMa
1,30438,45348,2326,Canopus,6.399195,-52.695660,-0.62,Car
2,69673,124897,5340,Arcturus,14.261030,19.182410,-0.05,Boo
3,71683,128620,5459,Rigil Kentaurus,14.660765,-60.833976,-0.01,Cen
4,91262,172167,7001,Vega,18.615640,38.783692,0.03,Lyr


In [33]:
merge_df = filter_df.merge(cons_df, left_on='con', right_on='Abbreviations IAU')

log_df(merge_df)
# (118, 10)

(118, 10)


,hip,hd,hr,proper,ra,dec,mag,con,Abbreviations IAU,Constellation
0,32349,48915,2491,Sirius,6.752481,-16.716116,-1.44,CMa,CMa,Canis Major
1,30438,45348,2326,Canopus,6.399195,-52.695660,-0.62,Car,Car,Carina
2,69673,124897,5340,Arcturus,14.261030,19.182410,-0.05,Boo,Boo,Boötes
3,71683,128620,5459,Rigil Kentaurus,14.660765,-60.833976,-0.01,Cen,Cen,Centaurus
4,91262,172167,7001,Vega,18.615640,38.783692,0.03,Lyr,Lyr,Lyra


In [34]:
cat_df = merge_df.copy()
cat_df['Catalogue Entry'] = 'HIP ' + cat_df['hip'].astype(str)
cat_df['Name catalog'] = 'HIP'
cat_df['Alternative Entries'] = ('HD ' + cat_df['hd'].astype(str) + ', HR ' + cat_df['hr'].astype(str))

cat_df['ra'] = cat_df['ra'].apply(decToHMS)
cat_df['dec'] = cat_df['dec'].apply(lambda row: decToHMS(row, True))

cat_df.rename(columns={
    'proper': 'Familiar Name',
    'ra': 'Right Ascension',
    'dec': 'Declination',
    'hip': 'Name number',
    'mag': 'Magnitude'
}, inplace=True)


cat_df['Major Axis'] = pd.NA
cat_df['Minor Axis'] = pd.NA
cat_df['Surface Brightness'] = pd.NA
cat_df["Width (')"] = pd.NA
cat_df["Height (')"] = pd.NA
cat_df['Surface Brightness'] = pd.NA
cat_df['Type'] = 'Star'
cat_df['Type Category'] = 'stars'
cat_df['Notes'] = 'bright_named_stars'

cat_df = cat_df[catalog_columns]

log_df(cat_df)
# (118, 16)

(118, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
0,HIP 32349,"HD 48915, HR 2491",Sirius,06:45:08.93,-16:42:58.02,<NA>,<NA>,-1.44,HIP,32349,Star,stars,Canis Major,<NA>,<NA>,bright_named_stars
1,HIP 30438,"HD 45348, HR 2326",Canopus,06:23:57.10,-52:41:44.38,<NA>,<NA>,-0.62,HIP,30438,Star,stars,Carina,<NA>,<NA>,bright_named_stars
2,HIP 69673,"HD 124897, HR 5340",Arcturus,14:15:39.71,+19:10:56.68,<NA>,<NA>,-0.05,HIP,69673,Star,stars,Boötes,<NA>,<NA>,bright_named_stars
3,HIP 71683,"HD 128620, HR 5459",Rigil Kentaurus,14:39:38.75,-60:50:02.31,<NA>,<NA>,-0.01,HIP,71683,Star,stars,Centaurus,<NA>,<NA>,bright_named_stars
4,HIP 91262,"HD 172167, HR 7001",Vega,18:36:56.30,+38:47:01.29,<NA>,<NA>,0.03,HIP,91262,Star,stars,Lyra,<NA>,<NA>,bright_named_stars


In [35]:
cat_df.to_csv(stars_catalog_path, index=False)

## create solar system catalogue

In [37]:
df = pd.read_csv(solar_system_path)

log_df(df)
# (9, 9)

(9, 9)


,Apparent magnitude (V),Object,Name,Seen from...,Notes,type,category,catalog,order
0,-26.71,Sun,Sun,seen from Earth,NaN,Sun,moon_planets,-,9
1,-3.69,Moon,Moon,"lit by earthlight, reflecting earthshine seen ...",NaN,Moon,moon_planets,-,8
2,0.23,planet Mercury,Mercury,seen from Earth,mean brightness[32],Planet,moon_planets,-,1
3,-4.14,planet Venus,Venus,seen from Earth,mean brightness[32],Planet,moon_planets,-,2
4,0.71,planet Mars,Mars,seen from Earth,mean brightness[32],Planet,moon_planets,-,3


In [39]:
cat_df = df.copy()


cat_df.rename(columns={
    'Name': 'Catalogue Entry',
    'Apparent magnitude (V)': 'Magnitude', 
    'catalog': 'Name catalog',
    'order': 'Name number',
    'type': 'Type',
    'category': 'Type Category'
}, inplace=True)

cat_df['Alternative Entries'] = pd.NA
cat_df['Familiar Name'] = pd.NA
cat_df['Right Ascension'] = pd.NA
cat_df['Declination'] = pd.NA
cat_df['Major Axis'] = pd.NA
cat_df['Minor Axis'] = pd.NA
cat_df['Constellation'] = pd.NA
cat_df["Width (')"] = pd.NA
cat_df["Height (')"] = pd.NA
cat_df['notes'] = pd.NA
cat_df = cat_df[catalog_columns]

log_df(cat_df)
# (9, 9)

(9, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
0,Sun,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-26.71,-,9,Sun,moon_planets,<NA>,<NA>,<NA>,NaN
1,Moon,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-3.69,-,8,Moon,moon_planets,<NA>,<NA>,<NA>,NaN
2,Mercury,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.23,-,1,Planet,moon_planets,<NA>,<NA>,<NA>,mean brightness[32]
3,Venus,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-4.14,-,2,Planet,moon_planets,<NA>,<NA>,<NA>,mean brightness[32]
4,Mars,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.71,-,3,Planet,moon_planets,<NA>,<NA>,<NA>,mean brightness[32]


In [40]:
cat_df.to_csv(solar_system_catalog_path, index=False)

In [41]:
cat_df.to_json(solar_system_catalog_json_path, orient='records', inplace=True)

## create DSO catalogue

In [43]:
dso_df = pd.read_csv(dso_catalog_path)
log_df(dso_df)
# (764, 16)

(764, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
0,B 92,NaN,on northside of M24,18:15:06,-18:14:00.0,15.0,9.0,NaN,B,92,Dark Nebula,nebulae,Sagittarius,NaN,NaN,large_dso
1,C 9,NaN,Cave Nebula,22:57:54.0,+62:31:06,50.0,30.0,NaN,C,9,HII Ionized region,nebulae,Cepheus,50.0,10.0,large_dso
2,C 14,NaN,"Double Cluster, h & chi Persei",02:20:42.0,+57:08:15,50.0,50.0,NaN,C,14,Association of stars,stars,Perseus,30.0,30.0,large_dso
3,C 41,NaN,Hyades,04:26:54.0,+15:52:00,329.0,NaN,NaN,C,41,Open Cluster,clusters,Taurus,330.0,330.0,large_dso
4,C 99,NTH CS,"Coalsack Nebula, Coalsack Northern",12:31:19.0,-63:44:36,NaN,NaN,NaN,C,99,Dark Nebula,nebulae,Crux,300.0,300.0,large_dso


In [45]:

stars_df = pd.read_csv(stars_catalog_path)
log_df(stars_df)
# (118, 16)

(118, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
0,HIP 32349,"HD 48915, HR 2491",Sirius,06:45:08.93,-16:42:58.02,NaN,NaN,-1.44,HIP,32349,Star,stars,Canis Major,NaN,NaN,bright_named_stars
1,HIP 30438,"HD 45348, HR 2326",Canopus,06:23:57.10,-52:41:44.38,NaN,NaN,-0.62,HIP,30438,Star,stars,Carina,NaN,NaN,bright_named_stars
2,HIP 69673,"HD 124897, HR 5340",Arcturus,14:15:39.71,+19:10:56.68,NaN,NaN,-0.05,HIP,69673,Star,stars,Boötes,NaN,NaN,bright_named_stars
3,HIP 71683,"HD 128620, HR 5459",Rigil Kentaurus,14:39:38.75,-60:50:02.31,NaN,NaN,-0.01,HIP,71683,Star,stars,Centaurus,NaN,NaN,bright_named_stars
4,HIP 91262,"HD 172167, HR 7001",Vega,18:36:56.30,+38:47:01.29,NaN,NaN,0.03,HIP,91262,Star,stars,Lyra,NaN,NaN,bright_named_stars


In [46]:
dso_df.columns == stars_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [48]:
combine_df = pd.concat([dso_df, stars_df])
combine_df.drop_duplicates(inplace=True)
log_df(combine_df)
# (882, 16)

(882, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
0,B 92,NaN,on northside of M24,18:15:06,-18:14:00.0,15.0,9.0,NaN,B,92,Dark Nebula,nebulae,Sagittarius,NaN,NaN,large_dso
1,C 9,NaN,Cave Nebula,22:57:54.0,+62:31:06,50.0,30.0,NaN,C,9,HII Ionized region,nebulae,Cepheus,50.0,10.0,large_dso
2,C 14,NaN,"Double Cluster, h & chi Persei",02:20:42.0,+57:08:15,50.0,50.0,NaN,C,14,Association of stars,stars,Perseus,30.0,30.0,large_dso
3,C 41,NaN,Hyades,04:26:54.0,+15:52:00,329.0,NaN,NaN,C,41,Open Cluster,clusters,Taurus,330.0,330.0,large_dso
4,C 99,NTH CS,"Coalsack Nebula, Coalsack Northern",12:31:19.0,-63:44:36,NaN,NaN,NaN,C,99,Dark Nebula,nebulae,Crux,300.0,300.0,large_dso


In [49]:
combine_df.sort_values(by=['Name catalog', 'Name number'], inplace=True)

In [50]:
combine_df.to_json(dso_catalog_json_path, orient='records', inplace=True)

## create messier catalog

In [52]:
dso_df = pd.read_csv(dso_catalog_path, dtype={'Name number': pd.Int64Dtype()})
log_df(dso_df)
# (764, 16)

(764, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
0,B 92,NaN,on northside of M24,18:15:06,-18:14:00.0,15.0,9.0,NaN,B,92,Dark Nebula,nebulae,Sagittarius,NaN,NaN,large_dso
1,C 9,NaN,Cave Nebula,22:57:54.0,+62:31:06,50.0,30.0,NaN,C,9,HII Ionized region,nebulae,Cepheus,50.0,10.0,large_dso
2,C 14,NaN,"Double Cluster, h & chi Persei",02:20:42.0,+57:08:15,50.0,50.0,NaN,C,14,Association of stars,stars,Perseus,30.0,30.0,large_dso
3,C 41,NaN,Hyades,04:26:54.0,+15:52:00,329.0,NaN,NaN,C,41,Open Cluster,clusters,Taurus,330.0,330.0,large_dso
4,C 99,NTH CS,"Coalsack Nebula, Coalsack Northern",12:31:19.0,-63:44:36,NaN,NaN,NaN,C,99,Dark Nebula,nebulae,Crux,300.0,300.0,large_dso


In [55]:
df = dso_df.copy()
df = df[(df['Name catalog']=='M') & (df['Familiar Name'].notna()) & (df['Major Axis'] < 17)]
log_df(df)
# (19, 16)

(19, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
76,M 1,NaN,Crab Nebula,05:34:31.97,+22:00:52.1,8.0,4.0,8.4,M,1,Supernova remnant,nebulae,Taurus,NaN,NaN,small_dso
81,M 6,NGC 6405,Butterfly Cluster,17:40:20.75,-32:15:15.0,15.6,NaN,4.2,M,6,Open Cluster,clusters,Scorpius,25.0,25.0,large_dso
86,M 11,NaN,"Amas de l'Ecu de Sobieski,Wild Duck Cluster",18:51:05.99,-06:16:12.1,9.0,NaN,5.8,M,11,Open Cluster,clusters,Scutum,NaN,NaN,small_dso
88,M 13,NGC 6205,"Great Hercules Globular, Hercules Globular Clu...",16:41:41.63,+36:27:40.7,16.5,NaN,5.8,M,13,Globular Cluster,clusters,Hercules,16.6,16.6,large_dso
92,M 17,NGC 6618,"Checkmark Nebula,Lobster Nebula,Swan Nebula,om...",18:20:47.11,-16:10:17.5,12.6,NaN,7.0,M,17,Nebula,nebulae,Sagittarius,46.0,46.0,large_dso


In [56]:
df.to_json(demo_path, orient='records')

## QA catalog

In [58]:
df = pd.read_json(dso_catalog_json_path, orient='records', inplace=True)
log_df(df)
# (882, 16)

(882, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Name catalog,Name number,Type,Type Category,Constellation,Width ('),Height ('),Notes
0,B 92,None,on northside of M24,18:15:06,-18:14:00.0,15.0,9.0,NaN,B,92,Dark Nebula,nebulae,Sagittarius,NaN,NaN,large_dso
1,C 9,None,Cave Nebula,22:57:54.0,+62:31:06,50.0,30.0,NaN,C,9,HII Ionized region,nebulae,Cepheus,50.0,10.0,large_dso
2,C 14,None,"Double Cluster, h & chi Persei",02:20:42.0,+57:08:15,50.0,50.0,NaN,C,14,Association of stars,stars,Perseus,30.0,30.0,large_dso
3,C 41,None,Hyades,04:26:54.0,+15:52:00,329.0,NaN,NaN,C,41,Open Cluster,clusters,Taurus,330.0,330.0,large_dso
4,C 99,NTH CS,"Coalsack Nebula, Coalsack Northern",12:31:19.0,-63:44:36,NaN,NaN,NaN,C,99,Dark Nebula,nebulae,Crux,300.0,300.0,large_dso


In [59]:
df.groupby(['Type']).size()

Type
Association of stars                         9
Dark Nebula                                  2
Galaxy                                     227
Galaxy Pair                                  1
Globular Cluster                            83
HII Ionized region                          26
Nebula                                      39
Open Cluster                               303
Other classification (see object notes)      1
Planetary Nebula                            30
Reflection Nebula                            9
Star                                       118
Star cluster + Nebula                       27
Supernova remnant                            7
dtype: int64

Association of stars                         9
Dark Nebula                                  2
Galaxy                                     227
Galaxy Pair                                  1
Globular Cluster                            83
HII Ionized region                          26
Nebula                                      39
Open Cluster                               303
Other classification (see object notes)      1
Planetary Nebula                            30
Reflection Nebula                            9
Star                                       118
Star cluster + Nebula                       27
Supernova remnant                            7

In [60]:
large_sizes = ((df["Width (')"] >= 15) | (df["Height (')"] >= 15) | 
               (df['Major Axis'] >= 15) | (df['Minor Axis'] >= 15) )

small_sizes = ((((df["Width (')"] < 15) & (df["Width (')"] > 1)) | ((df["Height (')"] < 15) & (df["Height (')"] > 1))) | 
               (((df['Major Axis'] < 15) & (df['Major Axis'] > 1)) | ((df['Minor Axis'] < 15) & (df['Minor Axis'] > 1))))

tiny_sizes = (((df["Width (')"] < 1) & (df["Height (')"] < 1)) | 
               ((df['Major Axis'] < 1) & (df['Minor Axis'] < 1)) )

In [62]:
# bright and  large
df[(df['Magnitude'] <= 10) & large_sizes].shape
# (119, 16)

(119, 16)

In [64]:
# unknown Magnitude and large
df[df['Magnitude'].isna() & large_sizes].shape
# (84, 16)

(84, 16)

In [66]:
# bright and small
df[small_sizes].shape
# (640, 16)

(640, 16)

In [68]:
# bright and  tiny
df[tiny_sizes].shape
# (2, 16)

(2, 16)

In [70]:
df[(df['Notes'] == 'tiny_dso')].shape
# (166, 16)

(166, 16)

In [72]:
df[(df['Notes'] == 'small_dso')].shape
# (364, 16)

(364, 16)

In [74]:
df[(df['Notes'] == 'large_dso')].shape
# (234, 16)

(234, 16)

In [75]:
# stars

df[(df['Type'] == 'Star') ].shape
# (118, 16)

(118, 16)

In [76]:
#  tiny + small_dso, large_dso , 'Star'

166 + 364 + 234 + 118

882

In [77]:
# bright + large, unknown mag and large

119 + 84

203